<a href="https://colab.research.google.com/github/LeroyMichael/Big_Data_Processing/blob/master/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when 
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
spark = SparkSession.builder.getOrCreate() 

In [ ]:
df_train = spark.read.option("inferSchema", "true").csv("/content/drive/My Drive/Classification/Planet_Training.csv", inferSchema=True,header=True)
df_train = df_train.select("Water", "Temperature", "Atmosphere Color", "Habitable")
df_train = df_train.na.drop()

In [ ]:
df_test = spark.read.option("inferSchema", "true").csv("/content/drive/My Drive/Classification/Planet_Testing.csv", inferSchema=True,header=True)
df_test = df_test.select("Water", "Temperature", "Atmosphere Color", "Habitable")
df_test = df_test.na.drop()

In [ ]:
def parse(df):
  df = df.withColumn("Water", when (df["Water"]=="Low", 0).
                     when(df["Water"]=="Medium", 1).
                     otherwise(2))
  df = df.withColumn("Atmosphere color", when (df["Atmosphere color"]=="Red", 0).
                     when(df["Atmosphere color"]=="Blue", 1).
                     otherwise(2))
  cols = df.columns
  cols.remove("Habitable")
  df = VectorAssembler(inputCols = cols, outputCol = "Features").transform(df)
  scaler = StandardScaler(inputCol = "Features", outputCol = "Scaled_Features")
  df = scaler.fit(df).transform(df)
  return df

In [ ]:
df_train=parse(df_train)
df_test=parse(df_test)

In [ ]:
model = LogisticRegression(featuresCol = "Scaled_Features", labelCol = "Habitable", maxIter=10).fit(df_train)

prediction = model.transform(df_test)

evaluator = BinaryClassificationEvaluator(labelCol = "Habitable")
acc = evaluator.evaluate(prediction) * 100
print("Accuracy : {}%".format(acc))


Accuracy : 91.71043337232418%
